In [1]:
import numpy as np
import matplotlib.pyplot as plt
import MSMRD as mrd
import pickle
import cython
%matplotlib inline

In [2]:
traj = np.loadtxt('../data/2DmodifiedLJmultipleTrajsLongR4.txt')

In [3]:
print traj.shape
trajs = []
for i in range(0, 10):
    trajs.append(traj[:,2*i:2*i+2])

(1000000, 20)


In [4]:
ratios = np.zeros(10)
k=0
radiusA = 1.0
radiusB = 2.3
bs=[]
for t in trajs:
    b = np.ones(len(t), dtype=float)
    absTraj = np.linalg.norm(t, axis=1)
    if absTraj[0] <= radiusA:
        b[0] = 0
    else:
        b[0] = 1
    for j in range(1, len(t)):
        if b[j-1] == 1:
            if absTraj[j] <= radiusA:
                b[j] =  0
            else:
                b[j] = 1
        elif b[j-1] == 0:
            if absTraj[j] >= radiusB:
                b[j] = 1
            else:
                b[j] = 0
    bs.append(b)

In [5]:
AtoBs = []
BtoAs = []
for b in bs:
    transitions = b[1:]-b[:-1]
    AtoB = np.where(transitions == 1)[0]
    BtoA = np.where(transitions == -1)[0]
    AtoBs.append(AtoB)
    BtoAs.append(BtoA)

In [6]:
def transitionTimes(AtoB, BtoA):
    if len(AtoB) == len(BtoA):
        if AtoB[0] < BtoA[0]:
            print "case1"
            tauBtoA = BtoA - AtoB
            tauAtoB = AtoB[1:] - BtoA[:-1]
        else:
            print "case2"
            tauAtoB = AtoB - BtoA
            tauBtoA = BtoA[1:] - AtoB[:-1]
    elif len(AtoB)+1 == len(BtoA):
        if AtoB[0] < BtoA[0]:
            print "case3"
            tauBtoA = BtoA[:-1] - AtoB
            tauAtoB = AtoB[1:] - BtoA[:-2]
        else:
            print "case4"
            tauAtoB = AtoB - BtoA[:-1]
            tauBtoA = BtoA[1:] - AtoB
    elif len(AtoB)-1 == len(BtoA):
        if AtoB[0] < BtoA[0]:
            print "case5"
            tauBtoA = BtoA - AtoB[:-1]
            tauAtoB = AtoB[1:] - BtoA
        else:
            print "case6"
            tauAtoB = AtoB[:-1] - BtoA
            tauBtoA = BtoA[1:-1] - AtoB[:-2]
    else:
        return None
    return tauAtoB, tauBtoA

In [7]:
tauAtoBs = []
tauBtoAs = []
for i in range(0, len(AtoBs)):
    tauAtoB, tauBtoA = transitionTimes(AtoBs[i], BtoAs[i])
    tauAtoBs.append(tauAtoB)
    tauBtoAs.append(tauBtoA)

case4
case2
case2
case2
case2
case2
case2
case2
case2
case2


In [8]:
timestep = 0.01
meanTauAtoB = [np.mean(tauAtoB*timestep) for tauAtoB in tauAtoBs]
meanTauBtoA = [np.mean(tauBtoA*timestep) for tauBtoA in tauBtoAs]
stdTauAtoB = [np.std(tauAtoB*timestep) for tauAtoB in tauAtoBs]
stdTauBtoA = [np.std(tauBtoA*timestep) for tauBtoA in tauBtoAs]
statDistrs = [np.sum(b)/float(len(b)) for b in bs]

In [9]:
timestep = 0.01
print 'Mean BtoA: ',np.mean(meanTauBtoA) 
print 'Mean AtoB: ',np.mean(meanTauAtoB) 
print 'Std BtoA: ',np.mean(stdTauBtoA)
print 'Std BtoA: ',np.mean(stdTauAtoB)
print 'Ratio: ',np.mean(meanTauBtoA)/np.mean(meanTauAtoB)
print 'Ratio: ',np.mean(stdTauBtoA)/np.mean(stdTauAtoB)
print 'Stat Distr: ' ,np.mean(statDistrs)

Mean BtoA:  7.29575442754
Mean AtoB:  1.10257840476
Std BtoA:  7.76817940468
Std BtoA:  0.864093863211
Ratio:  6.61699376303
Ratio:  8.98997173271
Stat Distr:  0.8686724
